In [1]:
import torch
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import numpy as np
from datasets import load_dataset, Dataset
from tqdm.notebook import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction, TrainingArguments, Trainer, pipeline
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [3]:
dataset=load_dataset('valurank/offensive-multi')

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['source', 'text', 'label'],
        num_rows: 36593
    })
})


In [5]:
dataset=dataset.filter(lambda x: False if x['source']=='olid' else True)

In [6]:
print (dataset)

DatasetDict({
    train: Dataset({
        features: ['source', 'text', 'label'],
        num_rows: 23353
    })
})


In [7]:
dataset=dataset.remove_columns('source')
#dataset=dataset.map(lambda x: {'label': 'off'} if x['label']==1 else {'label': 'not'})

In [8]:
off, nt = 0, 0
def count_ds(dataset):
    global off
    global nt
    off, nt = 0, 0
    for x in tqdm(dataset['train']):
        if x['label']==1:
            off+=1
        else:
            nt+=1
    print (off, nt)
count_ds(dataset)

  0%|          | 0/23353 [00:00<?, ?it/s]

19190 4163


In [9]:
contor=0
def filter_excess(x):
    global contor
    global nt
    if x['label']==0:
        return True
    else:
        if contor<nt:
            contor+=1
            return True
        else:
            return False
dataset=dataset.filter(filter_excess)
count_ds(dataset)

  0%|          | 0/8326 [00:00<?, ?it/s]

4163 4163


In [10]:
seed=42
dataset=dataset.shuffle(seed=seed)
dataset=Dataset.train_test_split(dataset['train'], test_size=0.1)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7493
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 833
    })
})


In [11]:
labels=['off', 'not']
label2id={
    'not': 0,
    'off': 1
}
id2label={
    0: 'not',
    1: 'off'
}

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  text = examples["text"]
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  encoding["labels"] = [[0.0, 1.0] if label==1 else [1.0, 0.0] for label in examples['label']]
  return encoding

encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/7493 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

In [13]:
print (encoded_dataset['train'][0])

{'input_ids': [101, 2017, 2024, 2025, 1037, 4937, 1012, 2017, 2024, 1037, 22418, 999, 2304, 11471, 4845, 8563, 1037, 8872, 2011, 3225, 1037, 2543, 1012, 2073, 2003, 1996, 19006, 2017, 6887, 16585, 1029, 999, 1029, 999, 1029, 999, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [14]:
torch.cuda.empty_cache()
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [16]:
encoded_dataset.set_format("torch")

In [17]:
batch_size = 30
metric_name = "f1"
epoch_count = 5 #TODO: Change this to 5 after testing

In [18]:
model_path=f"bert-offensive-finetuned"

In [19]:
#forward pass
outputs = model(
    input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), 
    labels=encoded_dataset['train'][0]['labels'].unsqueeze(0)
)
outputs

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


SequenceClassifierOutput(loss=tensor(0.6726, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[0.1027, 0.1974]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [20]:
args = TrainingArguments(
    model_path,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch_count,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #label_names=labels,
    #push_to_hub=True,
)

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.124884,0.962785,0.962785,0.962785
2,0.176800,0.132337,0.959760,0.959784,0.959184
3,0.176800,0.168324,0.962785,0.962785,0.962785
4,0.050400,0.202723,0.954382,0.954382,0.954382
5,0.050400,0.206569,0.956783,0.956783,0.956783


TrainOutput(global_step=1250, training_loss=0.09595350799560547, metrics={'train_runtime': 775.9862, 'train_samples_per_second': 48.28, 'train_steps_per_second': 1.611, 'total_flos': 2464363922265600.0, 'train_loss': 0.09595350799560547, 'epoch': 5.0})

In [23]:
trainer.evaluate()

{'eval_loss': 0.12488430738449097,
 'eval_f1': 0.9627851140456183,
 'eval_roc_auc': 0.9627851140456183,
 'eval_accuracy': 0.9627851140456183,
 'eval_runtime': 7.0971,
 'eval_samples_per_second': 117.371,
 'eval_steps_per_second': 3.945,
 'epoch': 5.0}

In [24]:
trainer.save_model(model_path)

In [25]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/alexsd06/bert-offensive-finetuned/commit/a894b087fe20dd63eb382d6058eb4e8fd6946231', commit_message='End of training', commit_description='', oid='a894b087fe20dd63eb382d6058eb4e8fd6946231', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
pipe = pipeline('text-classification', model=model_path, device='cuda', truncation=True)

In [48]:
pipe ("Go fuck off!")

[{'label': 'off', 'score': 0.9868566393852234}]

In [49]:
pipe ("Could you help me debug this problem?")

[{'label': 'not', 'score': 0.9847993850708008}]

In [29]:
idx='test'
l=len(dataset[idx])
good, bad = 0, 0
shown=0
for item in tqdm(dataset[idx]):
    text=item['text']
    label=item['label']
    label=id2label[label]
    predict=pipe(text)[0]['label']
    #if predict=='spam':
    #    print ('spam') #The model nevers predicts spam!!!
    #    break
    if predict==label:
        good+=1
    else:
        bad+=1

  0%|          | 0/833 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [30]:
print (f'Accuracy {100*good/l}%!')
print (f'Loss {100*bad/l}%!')

Accuracy 96.27851140456183%!
Loss 3.721488595438175%!


**5 Epochs:**
Accuracy 96.39855942376951%! Loss 3.6014405762304924%!

**1 Epochs:**
Accuracy 95.91836734693878%! Loss 4.081632653061225%!